<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 8 : Recommandation de films par filtrage collaboratif

## Résumé

Le but de ce laboratoire est d'entraîner et de tester plusieurs méthodes de recommandation par filtrage collaboratif :
* plusieurs *baselines* (notes moyennes par utilisateur ou par film)
* deux approches *memory-based* (modèle utilisateur-utilisateur ou film-film)
* une approche *model-based* (réduction de dimensionnalité)

Un jeu de données d'entraînement est fourni, ainsi qu'un jeu de validation, sur lequel vous pourrez tester vos méthodes et choisir les paramètres donnant les meilleurs résultats.

**Pour rendre ce travail,** veuillez répondre aux questions en écrivant le code demandé, résumer les tests effectués, et comparer entre eux les scores obtenus.  Veuillez ensuite rendre le notebook sur Cyberlearn.

**Pour participer à la compétition**, veuillez rendre séparément sur Cyberlearn un notebook qui permet d'exécuter seulement votre meilleure méthode sur les données de validation (nous les remplacerons par celles de test) ; veuillez rendre également le modèle qui est utilisé par votre notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error # fournit aussi la RMSE
from tqdm import tqdm # permet d'afficher la progression de l'exécution

## 1. Analyse exploratoire des données

Le groupe de recherche GroupLens met à disposition le [jeu de données MovieLens](http://files.grouplens.org/datasets/movielens/) contenant 20 millions de *ratings*, c'est-à-dire des notes données à des films par des utilisateurs.  Pour simplifier, deux extraits des données sont fournis dans les fichiers `ratings_train.csv` et `ratings_valid.csv` sur Cyberlearn.  

Ces fichiers contiennent les index originaux des utilisateurs et des films, ainsi que des index nouveaux (*new*) qui vont de 1 à `n_users` et de 1 à `n_movies` sans discontinuer. 

Note : GroupLens fournit aussi d'autres informations, notamment une correspondance entre les index originaux des films et leurs titres.  En revanche, les utilisateurs sont toujours anonymisés et sont désignés par leurs index.

**1a.** Veuillez charger les deux extraits de données dans deux *dataframes* appelées `train_df` et `valid_df`.
* Veuillez afficher les 3 premières lignes de chacune.
* Combien de *ratings* contient chacune des *dataframes* ?
* Quels sont les valeurs minimale, maximale, et moyenne des *ratings* dans chaque *dataframe* ?

**1b. Vérifications.** Quelles sont les plus petites et les plus grandes valeurs des *nouveaux* index des utilisateurs et des films, dans chacune des deux dataframes ?  Combien de valeurs différentes y a-t-il ? Est-ce qu'il y a des index manquants ?

**1c.** Veuillez d'abord définir les variables `n_users` et `n_movies` qui stockent le nombre d'utilisateurs et de films différents.  Quel est le nombre moyen de *ratings* par utilisateur ?  Et par film ?

Dans `train_df`, combien de *ratings* possède le film le plus souvent évalué ?  (Autrement dit, quel est le nombre maximal de *ratings* d'un film ?)  Combien de *ratings* possède le film le moins souvent évalué ?  

Indication : pour calculer le nombre de ratings par film, appliquer `groupby()` puis `count()` sur la *dataframe*. 

Dans `train_df`, combien de *ratings* a formulé l'utilisateur qui s'est le plus souvent exprimé ?  (Autrement dit, quel est le nombre maximal de *ratings* donnés par un utilisateur ?)  Combien de *ratings* a formulé l'utilisateur qui s'est le moins souvent exprimé ?

In [ ]:
# n_users =
# n_movies =


**1d (facultatif).** Le but ici est de mieux étudier la distribution des *ratings* par utilisateur, dont vous venez de calculer les valeurs moyenne, minimale et maximale.

Veuillez afficher (côte à côte si possible) les histogrammes du nombre de *ratings* par utilisateur dans `train_df` et dans `valid_df`.  Pensez à bien écrire les légendes des axes. Qu'observe-t-on en comparant `train_df` et dans `valid_df`?

Même question pour le nombre de *ratings* par film.

**1e.** Veuillez générer les matrices *utilisateurs x films* qui contiennent dans chaque cellule \[i, j\] le *rating* de l'utilisateur *i* pour le film *j*.  Si ce *rating* n'existe pas, la cellule vaut zéro.  Veuillez générer :
- la matrice `train_matrix` à partir de `train_df`
- la matrice `valid_matrix` à partir de `valid_df`

Quel est le taux de remplissage (cellules non nulles) de chaque matrice ?  On peut le calculer soit à partir des *dataframes*, soit directement sur les matrices.  Que pouvez-vous observer en comparant ces deux valeurs ?

## 2. Évaluation des systèmes "baseline"

Dans cette section, vous allez créer plusieurs systèmes *baseline* pour la comparaison, calculer leurs scores sur `valid_matrix` et les afficher.  Le calcul des scores vous est montré au point 2a.  Ces systèmes sont :
- **2a.** Prédiction des *ratings* comme valeurs aléatoires entre 1 et 10 (**code fourni**).
- **2b.** Prédiction des *ratings* par la moyenne de tous les *ratings* fournis dans `train_matrix`.
- **2c.** Prédiction des *ratings* d'un utilisateur par la moyenne de ses *ratings* fournis dans `train_matrix`.
- **2d.** Prédiction des *ratings* d'un film par la moyenne de ses *ratings* fournis dans `train_matrix`.
- **2e.** Prédiction de votre choix, par une combinaison des éléments de (b), (c) et (d).

In [ ]:
# 2a. Voici comment calculer la Root Mean Squared Error (RMSE) des prédictions par rapport aux ratings de valid_matrix.

valid_vector = valid_matrix[valid_matrix.nonzero()] # suite des ratings non-nuls de valid_matrix

predict_2a = np.random.randint(1, 11, (len(valid_vector), 1)) # ratings aléatoires pour les utilisateurs x films 

rmse_2a = mean_squared_error(valid_vector, predict_2a, squared=False) # 'False' pour avoir la RMSE

print(f'RMSE en faisant des prédictions aléatoires entre 1 et 11 : {round(rmse_2a, 4)}')

In [ ]:
# 2b -- veuillez écrire votre code ici et afficher le score RMSE
# 


In [ ]:
# 2c -- veuillez écrire votre code ici et afficher le score RMSE
# 


In [ ]:
# 2d -- veuillez écrire votre code ici et afficher le score RMSE
# 


In [ ]:
# 2e -- veuillez écrire votre code ici et afficher le score RMSE
# 


**2f.** Veuillez recopier ici les scores obtenus et commenter leurs différences.

## 3. Filtrage collaboratif basé sur les exemples : modèle utilisateur-utilisateur

Dans cette partie, vous allez implémenter et tester les modèles de filtrage collaboratif *"memory-based"* qui calculent les *ratings* prédits pour chaque utilisateur selon les films les plus appréciés par les utilisateurs semblables.  Les formules pour ce modèle ont été données en cours.  Il faudra faire attention aux sous-ensembles sur lesquels sont calculées les différentes sommes.

**3a.** Veuillez calculer les *ratings* moyens de chaque utilisateur et les stocker dans un tableau nommé `mean_user_ratings`.

Attention, il ne faut pas inclure les valeurs nulles dans le calcul de ces moyennes.  Les zéros ne sont pas de vrais scores (*ratings*) mais indiquent l'absence de score.  Si on les incluait, les moyennes seraient toutes très basses.

In [ ]:
# mean_user_ratings = ...


**3b.** *Memory-based collaborative filtering: user-user model.*  Veuillez implémenter la formule vue en cours pour calculer la matrice `sim`.  Il s'agit d'un coefficient de similarité entre deux utilisateurs, sur la base des *ratings* qu'ils ont formulés pour les films qu'ils ont jugés en commun.  On utilise la corrélation de Pearson, mais restreinte aux films communs, c'est-à-dire ceux avec des *ratings* non nuls des deux utilisateurs.  Pour cette raison, on ne peut pas appliquer directement la fonction `numpy.corrcoef()` sur `train_matrix`, mais on doit effectuer le calcul explicitement.

In [ ]:
# Initialisation de la matrice des similarités entre utilisateurs :
# sim[i, j] mesure la similarité entre les utilisateurs i et j.
# La matrice sera symétrique car sim[i, j] = sim[j, i], et elle peut
# avoir la diagonale nulle, car le modèle n'utilisera pas sim[i, i].

sim = np.zeros((n_users, n_users)) 

# Parcourir les ratings des utilisateurs i et j (avec i<j), c'est-à-dire 
# train_matrix[i] et train_matrix[j], retenir seulement les positions où
# les deux ratings sont >0, puis calculer le coefficient de corrélation.

for i in tqdm(range(n_users)):
    for j in range(i+1, n_users):


In [ ]:
# Une fonction auxiliaire utile.  Elle retourne une copie du vecteur 
# qui garde ses n plus grandes valeurs et met les autres à zéro.
def highest(array, n):
    if n >= len(array) or n<0:
        return array.copy()
    else:
        threshold = sorted(array, reverse=True)[n-1]
        result = array.copy()
        result[result<threshold]=0
        return result

In [ ]:
# Test de la fonction : retourne une copie de arr1 qui garde seulement
# ses 2 plus grands éléments, puis la même chose mais en gardant les
# deux plus grands éléments qui se trouvent à des emplacements non nuls
# de arr2 (cet exemple servira au 3c).
arr1 = np.array([1, 2, 5, 0, 9, 7])
arr2 = np.array([3, 0, 2, 0, 0, 7])
print(arr1, highest(arr1, 2), arr1)
print(arr1, highest(arr1 * np.where(arr2 > 0, 1, 0), 2), arr1)

**3c.** Veuillez calculer les *ratings* prédits par le système pour tous les utilisateurs et les items (comme au 2c et 2d mais chaque cellule doit être calculée avec la formule vue en cours).  Essayez d'optimiser le nombre d'utilisateurs semblables que vous retenez dans la formule.

In [ ]:
predict_3 = np.zeros((n_users, n_movies)) 


**3d.** Veuillez évaluer les *ratings* prédits en les comparant avec les *ratings* fournis par `valid_vector` calculé plus haut.  Comment se compare le résultat avec les scores du 2f ?

## 4. Filtrage collaboratif basé sur les exemples : modèle film-film

Dans cette partie, vous allez implémenter et tester les modèles de filtrage collaboratif *"memory-based"* qui calculent les *ratings* prédits pour un film selon les utilisateurs qui ont le plus apprécié les films semblables (modèle réciproque du précédent).  Les formules pour ce modèle ont été données en cours.  Il faudra faire attention aux sous-ensembles sur lesquels sont calculées les différentes sommes.

**4a.** Veuillez calculer les *ratings* moyens de chaque film et les stocker dans un tableau nommé `mean_movie_ratings`.  

Attention, il ne faut pas inclure les valeurs nulles dans le calcul de ces moyennes.  Les zéros ne sont pas de vrais scores (*ratings*) mais indiquent l'absence de score.

In [ ]:
# mean_item_ratings = ...


**4b.** *Memory-based collaborative filtering: item-item model.*  Veuillez implémenter la formule vue en cours pour calculer la matrice `sim_item`.  Il s'agit d'un coefficient de similarité entre deux films (items), sur la base des *ratings* qu'ils ont reçus des utilisateurs qui se sont exprimés sur les deux films.  On utilise la corrélation de Pearson, mais restreinte aux utilisateurs communs, c'est-à-dire ceux avec des *ratings* non nuls des deux films.  Pour cette raison, on ne peut pas appliquer directement la fonction `numpy.corrcoef()` sur `train_matrix`, mais on doit effectuer le calcul explicitement.  Le code est très similaire au point 3b.

In [ ]:
# Initialisation de la matrice des similarités entre films :
# sim_item[i, j] mesure la similarité entre les films i et j.
# La matrice sera symétrique car sim[i, j] = sim[j, i], et elle peut
# avoir la diagonale nulle, car le modèle n'utilisera pas sim[i, i].
sim_item = np.zeros((n_movies, n_movies)) 

# Parcourir les ratings des films i et j (avec i<j), c'est-à-dire 
# train_matrix[:,i] et train_matrix[:,j], retenir seulement les positions où
# les deux ratings sont >0, puis calculer le coefficient de corrélation.

for i in tqdm(range(n_movies)):
    for j in range(i,n_movies):


**4c.** Veuillez calculer les *ratings* prédits par le système pour tous les utilisateurs et les items (analogue au 3c).  Essayez d'optimiser le nombre d'films semblables que vous utilisez dans la formule.

In [ ]:
predict_4 = np.zeros((n_users, n_movies))


**4d.** Veuillez évaluer les *ratings* prédits en les comparant avec les *ratings* fournis par `valid_vector` calculé plus haut.  Comment se compare le résultat avec les scores du 2f et ceux du 3d ?

## 5. Filtrage collaboratif par factorisation de matrices

Dans cette section, vous allez mettre en place un modèle de prédiction des *ratings* utilisant la factorisation de matrices, comme vu en cours.  Vous calculerez deux matrices U et V, telles que R = U.T x V où R est la matrice des *ratings* (U.T désigne la transposée de U).  Le calcul direct d'une SVD ne fonctionne pas, car la matrice R (ici, `train_matrix`) contient beaucoup de 0 qui signifient en réalité des *ratings* inconnus.  Vous utiliserez comme données d'entraînement directement celles de `train_df`. 

L'hyperparamètre à régler est la dimension réduite de U et de V, notée `n_latent_factors`.

Vous utiliserez une approche par descente de gradient, implémentée en Keras, qui utilisera seulement les *ratings* connus (non nuls) pour s'entraîner.  Après avoir calculé U et V, vous pourrez estimer toutes les valeurs inconnues de R, puisque R = U.T x V -- il suffira pour cela d'utiliser le modèle en mode prédiction.

Vous évaluerez le résultat en comparant ces estimations avec les *ratings* connus de `valid_df`. 

In [ ]:
import keras
from tensorflow.keras.layers import Input, Embedding, Flatten, dot
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

**5a.** Veuillez définir le modèle Keras qui prend en entrée l'index de l'utilisateur et l'index du film, et produit en sortie le *rating* estimé comme le produit scalaire des embeddings des entrées (*voir le cours*).  Finir par la compilation du modèle selon la ligne de code fournie.

In [ ]:
# Définition du modèle
# n_latent_factors = 

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RootMeanSquaredError()])

**5b.** Veuillez entraîner le modèle pendant un certain nombre d'époques et sauvegarder l'historique des scores.

**5c.** Veuillez afficher sur un graphique l'évolution de la RMSE sur les données de validation (`valid_df`) au cours de l'entraînement.  Veuillez afficher aussi séparément la meilleure valeur de RMSE.  Après plusieurs expériences, veuillez laisser un graphique qui montre que l'apprentissage est satisfaisant et expliquer pourquoi. 

**5d.** Effectuez plusieurs expériences et décrivez-les brièvement ici, pour déterminer votre meilleur modèle -- c'est à dire les valeurs optimales de `n_latent_factors` et du nombre d'époques.  Utilisez comme critère la RMSE sur les données de validation.  Si ce modèle est meilleur que ceux du (3) ou du (4), veuillez le sauvegarder à l'aide de la fonction [save](https://keras.io/guides/serialization_and_saving/) de Keras pour le soumettre à la compétition.

**Fin du Labo 8.**  Veuillez nettoyer ce notebook, afficher les résultats et les commentaires demandés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.  

Ne pas oublier de soumettre également votre meilleur système avec le code nécessaire pour l'évaluer sur des données de test similaires à celles de validation.